<a href="https://colab.research.google.com/github/Nataliaricotorres/ean_PNL/blob/main/Tarea_de_modelos_de_conteo_%5BCountVectorizer%5D_%5BTfidfVectorizer%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea de modelos de conteo


---
INTEGRANTES
*  Geraldine Patiño
*  Natalia Rico
*  Juan Sebastian Lozano





# Tratamiento inicial data set
Se extrajeron los comentarios positivos y negativos basados en la calificacion dada por el usuario.
*  Positivo calificacion mayor o igual a 8
*  Negativo calificacion menor o igual a 7

Adicionalmente se identificaron comentarios en idiomas difrenetes al español y se clasifico la calidad de los comentarios segun su longitud siendo esta mayor a 60 caracteres.



In [ ]:
#Importacion de la data
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/Colab Notebooks/PNL'

filename = 'reviews_booking.csv'



Mounted at /content/drive


In [ ]:
import pandas as pd

df_reviews = pd.read_csv(path + '/' + filename)
df_reviews.head()
print(df_reviews.info())
print(df_reviews.columns)
#Pretratamiento de los datos
df_reviews2 = df_reviews.copy()
df_reviews2['Calificación'] = df_reviews['Calificación'].str.replace(',','.').astype(float)
df_reviews2.head()


<ipython-input-2-d52180da813d>:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv(path + '/' + filename)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438213 entries, 0 to 438212
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Unnamed: 0       438213 non-null  int64 
 1   País             436241 non-null  object
 2   Acomodación      19903 non-null   object
 3   Noches           438213 non-null  object
 4   Fecha hospedaje  438213 non-null  object
 5   Grupo viaje      438211 non-null  object
 6   Fecha reseña     438213 non-null  object
 7   Titulo           437552 non-null  object
 8   Calificación     438213 non-null  object
 9   Cosas Positivas  236856 non-null  object
 10  Cosas Negativas  180013 non-null  object
 11  reseña           438213 non-null  object
dtypes: int64(1), object(11)
memory usage: 40.1+ MB
None
Index(['Unnamed: 0', 'País', 'Acomodación', 'Noches', 'Fecha hospedaje',
       'Grupo viaje', 'Fecha reseña', 'Titulo', 'Calificación',
       'Cosas Positivas', 'Cosas Negativas', 'reseñ

,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
0,0,Venezuela,NaN,2 noches,julio de 2023,En pareja,31 de julio de 2023,Excepcional,10.0,Fue la Mejor Opción que pudimos haber tomado e...,NaN,Sí
1,1,Colombia,NaN,4 noches,octubre de 2024,Persona que viaja sola,23 de octubre de 2024,Fantástica,10.0,"Ubicación, comodidades y limpieza",NaN,Sí
2,2,Colombia,NaN,1 noche,octubre de 2024,Persona que viaja sola,15 de octubre de 2024,Excepcional,10.0,"El lugar esta muy bien ubicado, el apartamento...",NaN,Sí
3,3,Colombia,NaN,1 noche,agosto de 2024,En pareja,20 de agosto de 2024,Gran apartamento en una gran ubicación.,10.0,Todo estuvo perfecto. La ubicación y la relaci...,NaN,Sí
4,4,Colombia,NaN,3 noches,marzo de 2024,En pareja,27 de marzo de 2024,Excepcional,10.0,"Es muy cómodo y acogedor, la atención es excel...",NaN,Sí


In [ ]:
stats = df_reviews2['Calificación'].describe()
print(stats)

count    438213.000000
mean          8.401774
std           1.974704
min           1.000000
25%           8.000000
50%           9.000000
75%          10.000000
max          10.000000
Name: Calificación, dtype: float64


Teniendo en cuenta que el promedio de calificacion esta en 8,4, se toman como comentarios positivos aquellos por arriba de 8 y malos de 7 hacia abajo.

In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=7bba08bd22c188dfd58c3e207f07bf8f68776af5d93c47244d9d540a50b320db
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
#Separacion de la data en comentarios positivos y por paises

df_reviews_pos = df_reviews2[df_reviews2['Calificación']>= 8]#Filtro por calificacion
df_reviews_pos.head()

#Filtrar por paises
paises_suramerica = ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Perú', 'Surinam', 'Uruguay', 'Venezuela']
df_pos = df_reviews_pos[df_reviews_pos['País'].isin(paises_suramerica + ['Estados Unidos'])]['Cosas Positivas']




In [ ]:
from langdetect import detect
#Comentarios
df_pos = df_pos.fillna('')

df_pos_fin_1= [df_pos.iloc[i] for i in range(len(df_pos)) if df_pos.iloc[i] and len(df_pos.iloc[i]) > 60 and detect(df_pos.iloc[i]) == 'es']
df_pos_fin_1 = pd.Series(df_pos_fin_1).sample(7500)


DF_pos=pd.DataFrame()

DF_pos['Comentarios']=df_pos_fin_1
DF_pos['Clas']='Positivo'
DF_pos

,Comentarios,Clas
9429,La ubicación es excelente y el hotel está muy ...,Positivo
30507,Las habitaciones muy completas y amplias. El d...,Positivo
28514,"La habitación súper cómoda, siempre estaba lim...",Positivo
24931,Fueron muy amables conmigo durante mi estancia...,Positivo
8131,"Excelente atención, hasta tuve la oportunidad ...",Positivo
...,...,...
8846,El desayuno excelente y muy variado.\nEl servi...,Positivo
28313,"Excelentes instalaciones, muy comódas las habi...",Positivo
13033,"Exelente, cómodo, limpio y silencioso, excelen...",Positivo
24515,"El hotel es bonito, con buenas instalaciones y...",Positivo


In [ ]:
#Separacion de la data en comentarios negativos y por paises

df_reviews_neg = df_reviews2[df_reviews2['Calificación']<= 7]#Filtro por calificacion
df_reviews_neg




,Unnamed: 0,País,Acomodación,Noches,Fecha hospedaje,Grupo viaje,Fecha reseña,Titulo,Calificación,Cosas Positivas,Cosas Negativas,reseña
12,12,Colombia,NaN,1 noche,septiembre de 2023,Persona que viaja sola,19 de septiembre de 2023,Aceptable,5.0,La atención,Ventilador,Sí
19,19,Colombia,NaN,1 noche,octubre de 2024,En pareja,17 de octubre de 2024,Excelente ubicación,7.0,NaN,Realicé una reserva por bookin y me cobraron v...,Sí
23,23,Colombia,NaN,1 noche,agosto de 2023,Persona que viaja sola,22 de agosto de 2023,Bien,7.0,NaN,no pusierton toallas y no tienen agua caliente,Sí
25,25,Colombia,NaN,1 noche,mayo de 2023,Persona que viaja sola,17 de mayo de 2023,Bien,7.0,"Ubicación, perfecta si vas a hacer alguna dili...","Me cobraron un valor mayor, al que figuraba en...",Sí
32,32,Colombia,NaN,1 noche,septiembre de 2022,Persona que viaja sola,25 de septiembre de 2022,Bien,7.0,Buen desayuno,.,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...
438199,438199,Colombia,Apartamento de 1 dormitorio,8 noches,diciembre de 2023,Persona que viaja sola,26 de diciembre de 2023,Agradable,6.0,"Creo que el ambiente es bastante agradable,aun...",El.problema que tuve con un Duvet blanco que s...,Sí
438202,438202,Colombia,Apartamento de 1 dormitorio,1 noche,septiembre de 2024,Persona que viaja sola,12 de noviembre de 2024,Mal,3.0,NaN,"cucarachas en el apartamento, la anfitriona se...",Sí
438206,438206,Colombia,Estudio,1 noche,octubre de 2024,Persona que viaja sola,7 de octubre de 2024,Mal,3.0,NaN,me cambiaron de hotel y la habitacion que soli...,Sí
438207,438207,Ecuador,Estudio,2 noches,noviembre de 2024,En familia,6 de noviembre de 2024,Pésimo,1.0,NaN,No me atendieron,Sí


In [ ]:
df_reviews_neg['País'].unique()

array(['Colombia', 'Chile', 'Perú', 'Francia', 'Ecuador', 'Puerto Rico',
       'Alemania', 'Venezuela', 'Argentina', 'Italia', 'Estados Unidos',
       'Países Bajos', 'Uruguay', 'España', 'Panamá', 'Reino Unido',
       'Israel', 'Costa Rica', 'Eslovenia', 'Brasil', 'México',
       'República Dominicana', 'Nueva Zelanda', 'Canadá', 'Suecia',
       'Egipto', 'Australia', 'Dinamarca', 'Polonia', 'Aruba', 'Irlanda',
       'Suiza', 'Honduras', 'Rumanía', 'Luxemburgo', 'Bélgica', 'Bahréin',
       'Guatemala', 'India', 'Rusia', 'Grecia', nan, 'Austria',
       'Emiratos Árabes Unidos', 'Japón', 'China', 'El Salvador',
       'Portugal', 'Curaçao', 'Malta', 'Pakistán', 'Corea del Sur',
       'Croacia', 'Marruecos', 'Bolivia', 'Turquía', 'Bulgaria',
       'Hong Kong', 'Martinica', 'Islandia', 'Guadalupe', 'Estonia',
       'Finlandia', 'Eslovaquia', 'Jamaica', 'Noruega', 'Ucrania',
       'Hungría', 'Vietnam', 'Letonia', 'Serbia', 'Tailandia', 'Qatar',
       'República Checa', 'Belice

In [ ]:
#Filtrar por paises
paises_suramerica = ['Argentina', 'Bolivia', 'Brasil', 'Chile', 'Colombia', 'Ecuador', 'Guyana', 'Paraguay', 'Perú', 'Surinam', 'Uruguay', 'Venezuela']
df_neg = df_reviews_neg[df_reviews_neg['País'].isin(paises_suramerica + ['Estados Unidos'])]['Cosas Negativas']
df_neg


,Cosas Negativas
12,Ventilador
19,Realicé una reserva por bookin y me cobraron v...
23,no pusierton toallas y no tienen agua caliente
25,"Me cobraron un valor mayor, al que figuraba en..."
32,.
...,...
438197,"La falta de aseo mas frecuente, las toallas no..."
438199,El.problema que tuve con un Duvet blanco que s...
438202,"cucarachas en el apartamento, la anfitriona se..."
438206,me cambiaron de hotel y la habitacion que soli...


In [ ]:
#Data set con comentarios negativos
df_neg = df_neg.fillna('')

df_neg_fin_1= [df_neg.iloc[i] for i in range(len(df_neg)) if df_neg.iloc[i] and len(df_neg.iloc[i]) > 60 and detect(df_neg.iloc[i]) == 'es']
df_neg_fin_1 = pd.Series(df_neg_fin_1).sample(7500)

DF_neg=pd.DataFrame()

DF_neg ['Comentarios']=df_neg_fin_1
DF_neg ['Clas']='Negativo'
DF_neg

In [ ]:
DF_Final = pd.concat([DF_pos, DF_neg])
DF_Final

,Comentarios,Clas
9429,La ubicación es excelente y el hotel está muy ...,Positivo
30507,Las habitaciones muy completas y amplias. El d...,Positivo
28514,"La habitación súper cómoda, siempre estaba lim...",Positivo
24931,Fueron muy amables conmigo durante mi estancia...,Positivo
8131,"Excelente atención, hasta tuve la oportunidad ...",Positivo
...,...,...
7396,"Ruido del aire acondicionado, temperatura de l...",Negativo
21657,lo que no me gusto es el ruido de las camas y ...,Negativo
11067,"No había servicio de limpieza de habitación, l...",Negativo
14386,El apto reservado con dias de antelacion se en...,Negativo


In [ ]:
DF_Final.to_csv('DF_Final.csv')


# Modelo de conteo
## Preprocesamiento

In [ ]:
#Librerias requeridas
!python -m spacy download es_core_news_sm
!pip install unidecode
import unidecode
import pandas as pd
import numpy as np
import unicodedata ## Normalizacion de texto
import re ## Exprexiones regulares
import nltk ## Procesamiento de lenguaje natural
from nltk.corpus import stopwords ## Palabras vacias
from nltk.stem import SnowballStemmer ## Stemming
from nltk.tokenize import word_tokenize ## Tokenizacion
from nltk.tokenize import RegexpTokenizer ## Tokenizacion
from sklearn.feature_extraction.text import CountVectorizer ## Vectorizador
import spacy

############## Descarga de recursos de nltk ################
nltk.download('punkt') ## Tokenizador
nltk.download('stopwords') ## Palabras vacias
nltk.download('snowball_data') ## Stemming
nltk.download('wordnet') ## Lematizacion
nltk.download('punkt_tab')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 52.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package snowball_data to /root/nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
nlp = spacy.load("es_core_news_sm")
stopwords_esp = set(stopwords.words('spanish'))

def clean_text(text):
  # Validar texto
  if not isinstance(text, str) or not text.strip():
     return ''

  # Normalización
  text = text.lower()
  #text = unidecode.unidecode(text)

  # regex clean
  #text = re.sub(r'[^a-zA-Z0-9]', '', text)
  text = re.sub(r'[^\w\s]', '', text)  # Solo deja palabras y espacios
  text = re.sub(r'\d+', '', text)  # Eliminar números
  text = re.sub(r'[^\u0000-\u007F\u00C0-\u017F\s]', '', text)  # Filtrar caracteres no latinos


  # Tokenización
  tokens = word_tokenize(text)

  # Stop words
  tokens = [token for token in tokens if token not in stopwords_esp]

  ## Lemmatize

  doc = nlp(' '.join(tokens))
  tokens = [token.lemma_ for token in doc]

  return ' '.join(tokens)

In [ ]:
# Función para limpiar caracteres y eliminar tildes
def limpiar_caracteres_no_espanol_sin_tildes(texto):
    # Eliminar caracteres especiales y palabras en alfabetos no latinos
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto)  # Solo deja palabras y espacios
    texto = re.sub(r'[^\u0000-\u007F\u00C0-\u017F\s]', '', texto)  # Filtrar caracteres no latinos

    # Normalizar y eliminar tildes
    texto = unicodedata.normalize('NFD', texto)  # Descomponer caracteres acentuados
    texto = re.sub(r'[\u0300-\u036f]', '', texto)  # Eliminar marcas de acento (diacríticos)

    return texto


In [ ]:
df_limpio = pd.read_csv('/content/DF_Final.csv')


In [ ]:
df_clean=df_limpio.copy()
df_clean['Limpio'] = df_clean['Comentarios'].astype(str).apply(clean_text)
df_clean

,Unnamed: 0,Comentarios,Clas,Limpio
0,9429,La ubicación es excelente y el hotel está muy ...,Positivo,ubicación excelente hotel bien mantenido volve...
1,30507,Las habitaciones muy completas y amplias. El d...,Positivo,habitación completa amplio desayuno abundante ...
2,28514,"La habitación súper cómoda, siempre estaba lim...",Positivo,habitación súper cómoda siempre limpio desayun...
3,24931,Fueron muy amables conmigo durante mi estancia...,Positivo,amabl yo estancia agradecer atención servicio ...
4,8131,"Excelente atención, hasta tuve la oportunidad ...",Positivo,excelente atención oportunidad ir pasear doris...
...,...,...,...,...
14995,7396,"Ruido del aire acondicionado, temperatura de l...",Negativo,ruido aire acondicionado temperaturo habitació...
14996,21657,lo que no me gusto es el ruido de las camas y ...,Negativo,gusto ruido cama puerta baño
14997,11067,"No había servicio de limpieza de habitación, l...",Negativo,servicio limpiecer habitación parecer raro pue...
14998,14386,El apto reservado con dias de antelacion se en...,Negativo,apto reservado dias antelacion encontrar desas...


In [ ]:
#Crear una matriz de vectores
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df_clean['Limpio'])
df_vector = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
df_vector


,aa,aao,abajo,abandonado,abandonar,abandono,abandonáramos,abanico,abaratar,abarcar,...,óseo,óxido,ùtil,úber,úbicación,último,únicamente,único,ús,útil
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
14998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
